In [1]:
import os
 
def file_name(file_dir): 
    for root, dirs, files in os.walk(file_dir):
        continue
        #print(root) #当前目录路径
        #print(dirs) #当前路径下所有子目录
       # print(files) #当前路径下所有非目录子文件
    return files


## 控制c 只有需要检测的图片

# 加载模型

In [2]:
import argparse
import os
import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image

from myutils import rgb2ycbcr, ycbcr2rgb, calc_psnr

In [3]:
cudnn.benchmark = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
from rcan import make_model

In [5]:
class modelargs():
    def __init__(self):
        self.n_resgroups=10
        self.n_resblocks=20
        self.n_feats=64
        self.reduction=16
        self.scale=[2]
        self.rgb_range=255
        self.n_colors=3
        self.res_scale=1

In [6]:
model = make_model(modelargs()).to(device)

# 定义psnr函数

In [7]:
def get_psnr(testdata,myscale,picidx):
   # print(filename)
    scale=myscale
   # text= os.path.split(filename)[-1]
  #  textname = os.path.splitext(text)[0]
    
    lrpath="../LR/{}/x{}/".format(testdata,myscale)
    hrpath="../HR/{}/x{}/".format(testdata,myscale)
    
    clrpath=file_name(lrpath)
    chrpath=file_name(hrpath)
  #  print(clrpath[picidx])
    textname=str(clrpath[picidx]).split("_LRBI")[0]
#     textname=textnames[0]
#     for i in range(len(textnames)-1):
#         if(textnames[i+1]=='LRBI'):
#             break
#         textname=textname+textnames[i+1]
        
        
    
    
    
    lr = pil_image.open(lrpath+clrpath[picidx]).convert('RGB')
    lr = np.array(lr).astype(np.float32).transpose([2,0,1])
 #   lrycbcr = rgb2ycbcr(lr)
    
   # y = lrycbcr[..., 0]
   # print(lr.shape)
    y =lr /255.
    y = torch.from_numpy(y).to(device)
    y = y.unsqueeze(0)
    
    
    #print("y:{}".format(y.size()))########################################################################

    hr = pil_image.open(hrpath+chrpath[picidx]).convert('RGB')
    hr=np.array(hr).astype(np.float32).transpose([2,0,1])
  #  hrycbcrb = rgb2ycbcr(hr)
  #  ya = hrycbcrb[..., 0]
  
    ya =hr/ 255.
    ya = torch.from_numpy(ya).to(device)
    ya = ya.unsqueeze(0)
  #  print("ya:{}".format(ya.size()))########################################################################

    #print(y.size())

    with torch.no_grad():
        preds = model(y).clamp(0.0, 1.0)

    psnr = calc_psnr(ya, preds)

    print('PSNR: {:.2f}'.format(psnr))

    preds = preds.mul(255.0).cpu().numpy().squeeze(0)
    
    #print(preds.shape)
    output = np.array(preds).transpose([1, 2, 0])
    output = np.clip((output), 0.0, 255.0).astype(np.uint8)
    output = pil_image.fromarray(output)
    savepath='./{}/x{}/{}_myrcan_x{}.png'.format(testdata,myscale,textname,scale)
    
    if not os.path.exists(os.path.split(savepath)[0]):
        os.makedirs(os.path.split(savepath)[0])
     
    
    output.save(savepath)
    
    preds = None
    output = None
    ya = None
    y = None
    lr = None
    hr = None
    
    
    
    return psnr


# 单独计算psnr

In [8]:
modelname='./checkpoint/model_epoch_18.pth'
print(modelname)

./checkpoint/model_epoch_18.pth


In [9]:
testdata="Set5"
myscale = 2

In [10]:
hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)

In [11]:
count=len(clrpath)

In [12]:


hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)

5


In [13]:
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()

RCAN(
  (sf): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (rgs): Sequential(
    (0): RG(
      (module): Sequential(
        (0): RCAB(
          (module): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): ReLU(inplace=True)
            (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): ChannelAttention(
              (module): Sequential(
                (0): AdaptiveAvgPool2d(output_size=1)
                (1): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1))
                (2): ReLU(inplace=True)
                (3): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1))
                (4): Sigmoid()
              )
            )
          )
        )
        (1): RCAB(
          (module): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): ReLU(inplace=True)
            (2): Conv2d(64, 64, kernel_size=(3, 

In [14]:

sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

PSNR: 36.97
PSNR: 37.79
PSNR: 30.19
PSNR: 32.04
PSNR: 33.94
avg_psnr is :34.19


In [14]:

sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

PSNR: 23.71
PSNR: 23.82
PSNR: 20.26
PSNR: 25.62
PSNR: 23.54
avg_psnr is :23.39


In [17]:
allpsnr=[]

In [18]:
for j in range(1000):
    modelname='./checkpoint/model_epoch_{}.pth'.format(j+1)
    print(modelname)
    model.load_state_dict(torch.load(modelname))
    model=model.to(device)
    model.eval()

    testdata="Set5"
    myscale = 2
    hrpath="../HR/{}/x{}/".format(testdata,myscale)
    lrpath="../LR/{}/x{}/".format(testdata,myscale)
    clrpath=file_name(lrpath)
    count=len(clrpath)

    hrpath="../HR/{}/x{}/".format(testdata,myscale)

    chrpath=file_name(hrpath)

    count=len(chrpath)
    print(count)
    model=model.to(device)
    model.eval()
    sum_psnr=0
    for i in range(count):
     #   filename = os.path.join("../"+testdata+"/"+c[i])

        sum_psnr += get_psnr(testdata,myscale,i,j)

    print("avg_psnr is :{:.2f}".format(sum_psnr/count))
    allpsnr.append(sum_psnr/count)

./checkpoint/model_epoch_1.pth
5


TypeError: get_psnr() takes 3 positional arguments but 4 were given

In [15]:
testdata="Manga109"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

109
PSNR: 33.42
PSNR: 32.00
PSNR: 40.39
PSNR: 37.51
PSNR: 32.94
PSNR: 35.77
PSNR: 31.54
PSNR: 33.05
PSNR: 32.82
PSNR: 28.99
PSNR: 32.70
PSNR: 31.73
PSNR: 34.67
PSNR: 33.59
PSNR: 32.92
PSNR: 31.50
PSNR: 33.40
PSNR: 37.91
PSNR: 33.96
PSNR: 35.58
PSNR: 34.76
PSNR: 33.83
PSNR: 38.77
PSNR: 35.10
PSNR: 32.40
PSNR: 34.84
PSNR: 39.22
PSNR: 34.93
PSNR: 36.90
PSNR: 34.79
PSNR: 31.29
PSNR: 30.93
PSNR: 35.95
PSNR: 31.13
PSNR: 30.54
PSNR: 35.00
PSNR: 28.51
PSNR: 37.55
PSNR: 37.90
PSNR: 29.52
PSNR: 33.23
PSNR: 36.52
PSNR: 24.42
PSNR: 38.24
PSNR: 26.03
PSNR: 32.07
PSNR: 29.52
PSNR: 36.53
PSNR: 28.01
PSNR: 32.31
PSNR: 32.38
PSNR: 36.34
PSNR: 39.71
PSNR: 37.50
PSNR: 25.30
PSNR: 37.79
PSNR: 33.49
PSNR: 34.25
PSNR: 25.00
PSNR: 36.22
PSNR: 35.34
PSNR: 33.38
PSNR: 37.52
PSNR: 36.86
PSNR: 35.43
PSNR: 26.17
PSNR: 34.74
PSNR: 31.50
PSNR: 36.49
PSNR: 33.36
PSNR: 29.51
PSNR: 37.93
PSNR: 37.32
PSNR: 33.42
PSNR: 32.11
PSNR: 34.41
PSNR: 33.72
PSNR: 28.50
PSNR: 33.73
PSNR: 34.66
PSNR: 33.05
PSNR: 35.45
PSNR: 34.71


In [16]:
testdata="Set14"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

14
PSNR: 23.25
PSNR: 26.99
PSNR: 27.72
PSNR: 29.45
PSNR: 26.67
PSNR: 32.02
PSNR: 30.61
PSNR: 32.91
PSNR: 33.59
PSNR: 29.52
PSNR: 35.39
PSNR: 31.09
PSNR: 28.75
PSNR: 32.25
avg_psnr is :30.02


In [17]:
testdata="B100"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()

sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

100
PSNR: 25.45
PSNR: 30.33
PSNR: 28.04
PSNR: 33.54
PSNR: 28.51
PSNR: 35.97
PSNR: 30.19
PSNR: 25.40
PSNR: 30.36
PSNR: 32.21
PSNR: 27.13
PSNR: 31.73
PSNR: 33.78
PSNR: 31.54
PSNR: 27.54
PSNR: 29.43
PSNR: 36.39
PSNR: 34.69
PSNR: 27.27
PSNR: 27.86
PSNR: 24.92
PSNR: 26.60
PSNR: 26.09
PSNR: 28.83
PSNR: 30.92
PSNR: 30.19
PSNR: 30.36
PSNR: 32.82
PSNR: 34.12
PSNR: 21.51
PSNR: 32.89
PSNR: 22.91
PSNR: 28.61
PSNR: 26.15
PSNR: 37.30
PSNR: 29.62
PSNR: 28.67
PSNR: 29.54
PSNR: 31.75
PSNR: 38.31
PSNR: 29.12
PSNR: 27.46
PSNR: 28.75
PSNR: 30.80
PSNR: 26.45
PSNR: 36.25
PSNR: 25.54
PSNR: 27.45
PSNR: 30.57
PSNR: 33.90
PSNR: 26.62
PSNR: 26.08
PSNR: 33.66
PSNR: 34.23
PSNR: 30.96
PSNR: 28.16
PSNR: 22.55
PSNR: 29.22
PSNR: 33.54
PSNR: 32.23
PSNR: 33.30
PSNR: 30.99
PSNR: 37.55
PSNR: 26.87
PSNR: 28.12
PSNR: 31.21
PSNR: 41.76
PSNR: 23.78
PSNR: 24.43
PSNR: 32.33
PSNR: 32.01
PSNR: 28.16
PSNR: 29.35
PSNR: 28.02
PSNR: 26.42
PSNR: 32.54
PSNR: 25.55
PSNR: 32.49
PSNR: 34.39
PSNR: 31.92
PSNR: 38.34
PSNR: 29.62
PSNR: 24.20


In [18]:
testdata="Urban100"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)


model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

100
PSNR: 29.70
PSNR: 28.30
PSNR: 26.17
PSNR: 23.88
PSNR: 27.59
PSNR: 24.90
PSNR: 29.59
PSNR: 23.88
PSNR: 33.83
PSNR: 29.03
PSNR: 18.69
PSNR: 26.17
PSNR: 29.50
PSNR: 27.13
PSNR: 28.30
PSNR: 30.99
PSNR: 26.89
PSNR: 28.09
PSNR: 23.30
PSNR: 23.81
PSNR: 30.85
PSNR: 29.70
PSNR: 29.97
PSNR: 21.30
PSNR: 32.22
PSNR: 30.72
PSNR: 31.27
PSNR: 32.76
PSNR: 27.55
PSNR: 25.15
PSNR: 26.71
PSNR: 30.13
PSNR: 28.92
PSNR: 25.27
PSNR: 28.39
PSNR: 30.14
PSNR: 26.35
PSNR: 28.89
PSNR: 25.90
PSNR: 26.17
PSNR: 24.91
PSNR: 29.38
PSNR: 28.89
PSNR: 29.99
PSNR: 25.74
PSNR: 26.62
PSNR: 24.30
PSNR: 21.82
PSNR: 24.65
PSNR: 27.12
PSNR: 28.17
PSNR: 28.71
PSNR: 24.11
PSNR: 24.10
PSNR: 29.06
PSNR: 26.26
PSNR: 30.67
PSNR: 26.13
PSNR: 25.28
PSNR: 25.50
PSNR: 26.35
PSNR: 23.19
PSNR: 22.93
PSNR: 28.04
PSNR: 26.66
PSNR: 25.92
PSNR: 20.54
PSNR: 30.77
PSNR: 26.82
PSNR: 26.77
PSNR: 29.94
PSNR: 21.38
PSNR: 22.01
PSNR: 25.84
PSNR: 31.53
PSNR: 23.75
PSNR: 26.60
PSNR: 28.94
PSNR: 27.38
PSNR: 36.23
PSNR: 31.55
PSNR: 31.69
PSNR: 25.04


In [13]:
myscale = 3
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
   
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("psnr is :{:.2f}".format(sum_psnr/count))

PSNR: 30.63
PSNR: 25.66
PSNR: 34.00
PSNR: 33.36
PSNR: 27.44
PSNR: 28.82
PSNR: 27.34
PSNR: 27.23
PSNR: 26.26


KeyboardInterrupt: 

In [ ]:
myscale = 4
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
      
    sum_psnr += get_psnr(testdata,myscale,i)
print("psnr is :{:.2f}".format(sum_psnr/count))